In [1]:
import neuprint_functions
import numpy as np
import json
%load_ext autoreload
%autoreload 2 -p

In [2]:
mbon_id = 424789697 #MBON01_R
types, bodyids = neuprint_functions.get_all_neurons_presynaptic_to(mbon_id, supertype='KC')

In [5]:
synapses = neuprint_functions.pull_synapses(bodyids, post_id=mbon_id, n_iter=4)

  0%|          | 0/2324 [00:00<?, ?it/s]

  0%|          | 0/2351 [00:00<?, ?it/s]

In [7]:
# Factor to adjust from pixels to um coordinates
coordinates, lengths = neuprint_functions.pre_syn_coord_dict(synapses, bodyids)
print("The Total number of PRE>POST synapses is = "+str(len(synapses))) 
print("The total number of presynaptic KCs is = "+str(len(coordinates))) 
print("The average number of KC>MBON synapses per KC is = "+str(np.mean(lengths)))
coordinates_json = json.dumps(coordinates, indent = 4)
with open(f'synapse_coordinates_scaled_{mbon_id}.json', 'w') as output:
    output.write(coordinates_json)
output.close()

The Total number of PRE>POST synapses is = 9813
The total number of presynaptic KCs is = 1193
The average number of KC>MBON synapses per KC is = 8.225481978206203


In [9]:
synapses["pre_or_post"] = synapses["bodyId_pre"] == mbon_id
synapses["pre_or_post"][synapses["pre_or_post"] == True] = "pre"
synapses["pre_or_post"][synapses["pre_or_post"] == False] = "post"

/var/folders/lh/qs4dckx57_x56y2s1_8blvm80000gq/T/ipykernel_29369/2046546812.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  synapses["pre_or_post"][synapses["pre_or_post"] == True] = "pre"
/var/folders/lh/qs4dckx57_x56y2s1_8blvm80000gq/T/ipykernel_29369/2046546812.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  synapses["pre_or_post"][synapses["pre_or_post"] == False] = "post"
/var/folders/lh/qs4dckx57_x56y2s1_8blvm80000gq/T/ipykernel_29369/2046546812.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'post' has dtype incompatible with bool

In [12]:
synapses

,index,bodyId_pre,bodyId_post,roi_pre,roi_post,x_pre,y_pre,z_pre,x_post,y_post,z_post,confidence_pre,confidence_post,pre_or_post
0,0,5813091744,424789697,b'L(R),b'L(R),19671,30054,19484,19669,30031,19466,0.889,0.715545,post
1,1,5813091744,424789697,bL(R),bL(R),19769,30128,19705,19738,30112,19705,0.972,0.486082,post
2,2,5813091744,424789697,bL(R),bL(R),22468,28453,19981,22470,28469,19981,0.905,0.998535,post
3,3,5813091744,424789697,bL(R),bL(R),21553,28574,20531,21547,28558,20515,0.899,0.858935,post
4,4,5813091744,424789697,bL(R),bL(R),21700,28618,20412,21707,28631,20409,0.953,0.842463,post
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9808,2617,300968622,424789697,bL(R),bL(R),20531,31039,19793,20523,31050,19782,0.980,0.995538,post
9809,2618,300968622,424789697,bL(R),bL(R),20755,31076,19897,20749,31074,19883,0.957,0.889143,post
9810,2619,300968622,424789697,bL(R),bL(R),22452,29592,22245,22460,29593,22257,0.897,0.985913,post
9811,2620,300968622,424789697,bL(R),bL(R),23151,29723,21140,23151,29739,21147,0.957,0.855462,post


In [34]:
skel_graph = neuprint_functions.pull_skeleton(mbon_id, skel_format='nx')
skel = neuprint_functions.pull_skeleton(mbon_id, skel_format='pandas')
skel_swc, relabel_dict = neuprint_functions.relabel_skeleton_swc(skel_graph, skel)

Reloading 'neuprint_functions'.


In [ ]:
with open("neuron_skeleton.swc", "w") as output:
    output.write(skel_swc)